In [1]:
%load_ext autoreload
%autoreload 2

In [77]:
import torch
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from datasets import load_dataset, Dataset
from sibyl import *
import random
import pandas as pd

torch.use_deterministic_algorithms(False)

### Dataset

In [3]:
dataset = load_dataset("glue", "sst2", split="train")
dataset = dataset.rename_column("sentence", "text")
original_text, original_labels = dataset['text'], dataset['label']

Found cached dataset glue (C:/Users/Fabrice/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


### Model

In [4]:
pipe = pipeline(task="sentiment-analysis", device=0)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


### Transforms

In [90]:
blacklist = [Concept2Sentence, ConceptMix, Emojify]
ts = [t(task_name="sentiment", return_metadata=True) for t in TRANSFORMATIONS if t not in blacklist]

### Feature 

In [6]:
def contains_question(s):
    return "?" in s

### Analysis

In [7]:
from sibyl import acc_at_k
from sklearn.metrics import accuracy_score

In [8]:
def extract_probs(results):
    return np.array([[1-r['score'], r['score']] if r['label'] == "POSITIVE" else [r['score'], 1-r['score']] for r in results])

def compute_accuracy(predictions, labels):
    if len(labels.shape) > 1:
        acc = acc_at_k(labels, predictions, k=2)       
    else:
        acc = accuracy_score(labels, np.argmax(predictions, -1))
    return acc

In [169]:
# for trees 
#   if_change_on_leaves
#   change_to_X_node (where X is any one of the AMR nodes)
class BooleanFeature:
    def __init__(self, featurizer):
        self.featurizer = featurizer      
    
    def __call__(self, texts):
        return [self.featurizer(text) for text in texts]
    
    def extract_data_containing_feature(self, dataset):
        
        # extract T and F evaluation sets
        ts, fs = [], []
        for s, l in zip(dataset['text'], dataset['label']):
            if self.featurizer(s):
                ts.append((s, l))
            else:
                fs.append((s,l))

        t_texts, t_labels = zip(*ts)
        f_texts, f_labels = zip(*random.sample(fs, len(t_texts)))

        t_texts, t_labels = list(t_texts), one_hot_encode(t_labels, 2)
        f_texts, f_labels = list(f_texts), one_hot_encode(f_labels, 2)

        self.t_dataset = Dataset.from_list(
            [{'text': t, 'label': l} for t, l in zip(t_texts, t_labels)])
        self.f_dataset = Dataset.from_list(
            [{'text': t, 'label': l} for t, l in zip(f_texts, f_labels)])
            
    def generate_transformed_datasets(self, transform):
        
        self.transform = transform
         
        # apply transformation to each set of texts containing the target feature
        transformed_t_text, transformed_t_labels = transform.transform_batch(
            batch=(self.t_dataset['text'], self.t_dataset['label']))
        transformed_f_text, transformed_f_labels = transform.transform_batch(
            batch=(self.f_dataset['text'], self.f_dataset['label']))
        transformed_t_labels = np.stack([np.array(a).squeeze() for a in transformed_t_labels])
        transformed_f_labels = np.stack([np.array(a).squeeze() for a in transformed_f_labels])
        
        self.tran_t_dataset = Dataset.from_list(
            [{'text': t, 'label': l} for t, l in zip(transformed_t_text, transformed_t_labels)])
        self.tran_f_dataset = Dataset.from_list(
            [{'text': t, 'label': l} for t, l in zip(transformed_f_text, transformed_f_labels)]) 
        
        self.t_changed = sum([t1 != t2 for t1, t2 in zip(self.t_dataset, self.tran_t_dataset)]) / len(self.t_dataset['text'])
        self.f_changed = sum([t1 != t2 for t1, t2 in zip(self.f_dataset, self.tran_f_dataset)]) / len(self.f_dataset['text'])
    
    def evaluate_original(self, pipe):
        
        # pass data through the model
        self.orig_t_preds = extract_probs(pipe(KeyDataset(self.t_dataset, "text")))
        self.orig_f_preds = extract_probs(pipe(KeyDataset(self.f_dataset, "text")))
        
        # compute accuracy
        self.orig_t_acc = compute_accuracy(
            self.orig_t_preds, np.argmax(self.t_dataset['label'], -1))
        self.orig_f_acc = compute_accuracy(
            self.orig_f_preds, np.argmax(self.f_dataset['label'], -1))
        
        results = {
            "transform": "original",
            "featurizer": self.featurizer.__name__,
            "T_orig_acc": self.orig_t_acc,
            "F_orig_acc": self.orig_f_acc,
            "T_tran_acc": 0,
            "F_tran_acc": 0,
            "T_changed": 0,
            "F_changed": 0
        }
        
        return results
            
    def evaluate_transform(self, pipe):
               
        # pass data through the model
        self.tran_t_preds = extract_probs(pipe(KeyDataset(self.tran_t_dataset, "text")))
        self.tran_f_preds = extract_probs(pipe(KeyDataset(self.tran_f_dataset, "text")))
        
        # compute accuracy
        self.tran_t_acc = compute_accuracy(
            self.tran_t_preds, np.array(self.tran_t_dataset['label']))
        self.tran_f_acc = compute_accuracy(
            self.tran_f_preds, np.array(self.tran_f_dataset['label']))
        
        results = {
            "transform": self.transform.__class__.__name__,
            "featurizer": self.featurizer.__name__,
            "T_orig_acc": self.orig_t_acc,
            "F_orig_acc": self.orig_f_acc,
            "T_tran_acc": self.tran_t_acc,
            "F_tran_acc": self.tran_f_acc,
            "T_changed": self.t_changed,
            "F_changed": self.f_changed
        }
        
        return results
    
    def get_before_and_after(self):
        # Feature DataFrame: T
        t_df = pd.DataFrame(
            [
                f.t_dataset["text"], 
                f.t_dataset["label"],
                f.orig_t_preds,
                f.tran_t_dataset["text"],
                f.tran_t_dataset["label"],
                f.tran_t_preds,
            ]).T
        t_df.columns = ['orig_t_text', 'orig_t_label', 'orig_t_preds', 
                        'tran_t_text', 'tran_t_label', 'tran_t_preds']
        
        t_df['t_pred_diff'] = t_df.apply(lambda row: np.array(row['orig_t_preds']) - np.array(row['tran_t_preds']), axis=1)

        # Feature DataFrame: F
        f_df = pd.DataFrame(
            [
                f.f_dataset["text"], 
                f.f_dataset["label"],
                f.orig_f_preds,
                f.tran_f_dataset["text"],
                f.tran_f_dataset["label"],
                f.tran_f_preds,
            ]).T
        f_df.columns = ['orig_f_text', 'orig_f_label', 'orig_f_preds', 
                        'tran_f_text', 'tran_f_label', 'tran_f_preds']

        f_df['f_pred_diff'] = f_df.apply(lambda row: np.array(row['orig_f_preds']) - np.array(row['tran_f_preds']), axis=1)
        
        return t_df, f_df

In [177]:
f = BooleanFeature(contains_question)
f.extract_data_containing_feature(dataset)

results = []
results.append(f.evaluate_original(pipe))

for t in ts:
    f.generate_transformed_datasets(t)
    result = f.evaluate_transform(pipe)
    # t_df, f_df = f.get_before_and_after()
    # result['t_df'] = t_df
    # result['f_df'] = f_df
    results.append(result)

C:\Users\Fabrice\anaconda3\envs\dpml\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
C:\Users\Fabrice\anaconda3\envs\dpml\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [178]:
df = pd.DataFrame(results)

In [179]:
df['T_diff'] = df['T_orig_acc'] - df['T_tran_acc']
df['F_diff'] = df['F_orig_acc'] - df['F_tran_acc']

#### Observations

Note: Significant differences are any greater than 10%

1. Relative to a no-transform baseline, the presence of a question in a text leads to more stable behavior in sibyl transforms.
   - Twelve (12) transforms induce significant accuracy changes for texts not containing a question.
   - Only six (6) transforms induce significant accuracy changes for texts containing a question.  
2. Four (4) transforms exhibit significant differences in impact for different `contains_question` features.
3. Six (6) transforms did not apply to the sampled dataset.

In [193]:
# significant impact 
df[df['T_diff'] > 0.1]

,transform,featurizer,T_orig_acc,F_orig_acc,T_tran_acc,F_tran_acc,T_changed,F_changed,t_df,f_df,T_diff,F_diff
0,original,contains_question,1.0,0.994624,0.000000,0.000000,0.000000,0.000000,NaN,NaN,1.000000,0.994624
12,InsertPositivePhrase,contains_question,1.0,0.994624,0.586022,0.849462,1.000000,1.000000,ori...,ori...,0.413978,0.145161
28,ChangeAntonym,contains_question,1.0,0.994624,0.870968,0.650538,0.951613,0.876344,ori...,ori...,0.129032,0.344086
32,HomoglyphSwap,contains_question,1.0,0.994624,0.838710,0.510753,1.000000,1.000000,ori...,ori...,0.161290,0.483871
34,TextMix,contains_question,1.0,0.994624,0.897849,0.833333,1.000000,1.000000,ori...,ori...,0.102151,0.161290
35,SentMix,contains_question,1.0,0.994624,0.897849,0.822581,1.000000,1.000000,ori...,ori...,0.102151,0.172043
36,WordMix,contains_question,1.0,0.994624,0.811828,0.790323,1.000000,1.000000,ori...,ori...,0.188172,0.204301


In [194]:
df[df['F_diff'] > 0.1]

,transform,featurizer,T_orig_acc,F_orig_acc,T_tran_acc,F_tran_acc,T_changed,F_changed,t_df,f_df,T_diff,F_diff
0,original,contains_question,1.0,0.994624,0.000000,0.000000,0.000000,0.000000,NaN,NaN,1.000000,0.994624
12,InsertPositivePhrase,contains_question,1.0,0.994624,0.586022,0.849462,1.000000,1.000000,ori...,ori...,0.413978,0.145161
13,InsertNegativePhrase,contains_question,1.0,0.994624,0.935484,0.741935,1.000000,1.000000,ori...,ori...,0.064516,0.252688
17,AddNegativeLink,contains_question,1.0,0.994624,0.951613,0.881720,1.000000,1.000000,ori...,ori...,0.048387,0.112903
19,AddNegation,contains_question,1.0,0.994624,0.973118,0.870968,0.435484,0.322581,ori...,ori...,0.026882,0.123656
27,ChangeSynonym,contains_question,1.0,0.994624,0.908602,0.838710,0.951613,0.908602,ori...,ori...,0.091398,0.155914
28,ChangeAntonym,contains_question,1.0,0.994624,0.870968,0.650538,0.951613,0.876344,ori...,ori...,0.129032,0.344086
29,ChangeHyponym,contains_question,1.0,0.994624,0.924731,0.849462,0.956989,0.844086,ori...,ori...,0.075269,0.145161
30,ChangeHypernym,contains_question,1.0,0.994624,0.930108,0.876344,0.956989,0.887097,ori...,ori...,0.069892,0.118280
32,HomoglyphSwap,contains_question,1.0,0.994624,0.838710,0.510753,1.000000,1.000000,ori...,ori...,0.161290,0.483871


In [195]:
# significant impact difference by feature
df[abs(df['T_diff'] - df['F_diff']) > 0.1]

,transform,featurizer,T_orig_acc,F_orig_acc,T_tran_acc,F_tran_acc,T_changed,F_changed,t_df,f_df,T_diff,F_diff
12,InsertPositivePhrase,contains_question,1.0,0.994624,0.586022,0.849462,1.000000,1.000000,ori...,ori...,0.413978,0.145161
13,InsertNegativePhrase,contains_question,1.0,0.994624,0.935484,0.741935,1.000000,1.000000,ori...,ori...,0.064516,0.252688
28,ChangeAntonym,contains_question,1.0,0.994624,0.870968,0.650538,0.951613,0.876344,ori...,ori...,0.129032,0.344086
32,HomoglyphSwap,contains_question,1.0,0.994624,0.838710,0.510753,1.000000,1.000000,ori...,ori...,0.161290,0.483871


In [128]:
# inapplicable transforms
df[(df['T_changed'] == 0) & (df['F_changed'] == 0)] 

,transform,featurizer,T_orig_acc,F_orig_acc,T_tran_acc,F_tran_acc,T_changed,F_changed,T_diff,F_diff
0,original,contains_question,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
1,ExpandContractions,contains_question,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
6,Demojify,contains_question,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
7,RemovePositiveEmoji,contains_question,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
8,RemoveNegativeEmoji,contains_question,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
9,RemoveNeutralEmoji,contains_question,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
18,ImportLinkText,contains_question,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [198]:
# InsertPositivePhrase Details
df.iloc[12].t_df.iloc[0].orig_t_text

"... a sour little movie at its core ; an exploration of the emptiness that underlay the relentless gaiety of the 1920 's ... the film 's ending has a `` what was it all for ? '' "

In [202]:
df.iloc[12].t_df.iloc[0].orig_t_label

[1.0, 0.0]

In [199]:
df.iloc[12].t_df.iloc[0].tran_t_text

"... a sour little movie at its core ; an exploration of the emptiness that underlay the relentless gaiety of the 1920 's ... the film 's ending has a `` what was it all for ? ''  That being said, I loved it."

In [200]:
df.iloc[12].t_df.iloc[0].tran_t_label

[0.8640776699029126, 0.13592233009708743]

In [204]:
df.iloc[12].t_df.iloc[0].orig_t_preds

array([9.99622107e-01, 3.77893448e-04])

In [203]:
df.iloc[12].t_df.iloc[0].tran_t_preds

array([0.96844983, 0.03155017])